## SSH key pair generation

we create a SSH key pair that will be used to connect to the deployed node.

In [1]:
%x(ssh-keygen -b 2048 -t rsa -f /tmp/sshkey -q -N "")

"/tmp/sshkey already exists.\nOverwrite (y/n)? "

## Job and deployment definition

In [2]:
require 'xp5k'
myxp = XP5K::XP.new
myxp.define_job({
  :resources => ["{cluster='hercule'}/nodes=4, walltime=1:00:00"],
    :site      => 'lyon',
    :types     => ["deploy"],
    :name      => "iruby notebook" ,
    :roles     => [XP5K::Role.new({:name => 'mynode', :size => 4})],
    :command   => "sleep 86400"
    })
myxp.define_deployment({
    :site           => 'lyon',
    :environment    => "jessie-x64-nfs",
    :roles          => %w{ mynode },
    :key            => File.read("/tmp/sshkey.pub")
})

[{:site=>"lyon", :environment=>"jessie-x64-nfs", :roles=>["mynode"], :key=>"ssh-rsa AAAAB3NzaC1yc2EAAAADAQABAAABAQDNFSii9fyPUmHLl5wHGJ2CCLfV/kfL/9LnLROUFdAINbfC2MCF2u24tgQgJT4w2emegf+lKJZ4YL/nw7zvMVYGPF9tkRjB2L6v+7idXMR/r2Q+aL52dl2BzCkup34FUKLJtP6PPgWpo8VNSK9jgn8bnyTHIH6ZmTKp1ercAHSjs1se96czb24G68MQdz6YXJrlOrVeE9zhsEb95F29y+AmmhJp0SFaHHJ1J85j+QiamNdnCp5Ceqcu6qx2OgBK+k/aIeQUqtFsgWoenY+OaSajPS+ZLBZkrEgf1vJrBegCNYFCbjPI03xMjpzA2EfgDGltAe9dT8ODG4eUTX9hJqGN root@8b1d0aea7346\n", :jobs=>[]}]

## Job and deployment submission

In [3]:
myxp.submit
myxp.wait_for_jobs
myxp.deploy

....................... [OK]
Summary of the deployment
------------------------------------------------------------
                Name            Deployed          Undeployed

------------------------------------------------------------
              mynode                   4                   0



{"mynode"=>["hercule-1.lyon.grid5000.fr", "hercule-2.lyon.grid5000.fr", "hercule-3.lyon.grid5000.fr", "hercule-4.lyon.grid5000.fr"]}

## Install fping on all the nodes

In [4]:
require 'xp5k/rake'
h = on roles('mynode'), {:user => 'root', :ssh => {:keys => ['/tmp/sshkey']}} do 
  cmd = ['apt-get update -qq && apt-get install -qy fping']
end

Connected to hercule-1.lyon.grid5000.fr...
Connected to hercule-2.lyon.grid5000.fr...
Connected to hercule-3.lyon.grid5000.fr...
Connected to hercule-4.lyon.grid5000.fr...
[command][hercule-1.lyon.grid5000.fr] apt-get update -qq && apt-get install -qy fping
[command][hercule-2.lyon.grid5000.fr] apt-get update -qq && apt-get install -qy fping
[command][hercule-3.lyon.grid5000.fr] apt-get update -qq && apt-get install -qy fping
[command][hercule-4.lyon.grid5000.fr] apt-get update -qq && apt-get install -qy fping
[stdout][hercule-3.lyon.grid5000.fr] Reading package lists...
[stdout][hercule-3.lyon.grid5000.fr] Building dependency tree...
[stdout][hercule-2.lyon.grid5000.fr] Reading package lists...
[stdout][hercule-2.lyon.grid5000.fr] Building dependency tree...
[stdout][hercule-4.lyon.grid5000.fr] Reading package lists...
[stdout][hercule-3.lyon.grid5000.fr] 
Reading state information...
[stdout][hercule-4.lyon.grid5000.fr] Building dependency tree...
[stdout][hercule-2.lyon.grid5000.fr]

{"hercule-3.lyon.grid5000.fr"=>"Reading package lists...\nBuilding dependency tree...\nReading state information...\nThe following NEW packages will be installed:\n  fping\n0 upgraded, 1 newly installed, 0 to remove and 13 not upgraded.\nNeed to get 37.4 kB of archives.\nAfter this operation, 133 kB of additional disk space will be used.\nGet:1 http://ftp.debian.org/debian/ jessie/main fping amd64 3.10-2 [37.4 kB]\ndebconf: unable to initialize frontend: Dialog\ndebconf: (TERM is not set, so the dialog frontend is not usable.)\ndebconf: falling back to frontend: Readline\ndebconf: unable to initialize frontend: Readline\ndebconf: (This frontend requires a controlling tty.)\ndebconf: falling back to frontend: Teletype\ndpkg-preconfigure: unable to re-open stdin: \nFetched 37.4 kB in 0s (1,111 kB/s)\nSelecting previously unselected package fping.\r\n(Reading database ... \r(Reading database ... 5%\r(Reading database ... 10%\r(Reading database ... 15%\r(Reading database ... 20%\r(Reading 

## Each node pings all nodes

In [5]:
servers = roles('mynode')
h = on roles('mynode'), {:user => 'root', :ssh => {:keys => ['/tmp/sshkey']}} do 
  cmd = "cat > /tmp/hosts << EOF
#{servers.join("\n")}
EOF"
end

Connected to hercule-1.lyon.grid5000.fr...
Connected to hercule-3.lyon.grid5000.fr...
Connected to hercule-2.lyon.grid5000.fr...
Connected to hercule-4.lyon.grid5000.fr...
[command][hercule-1.lyon.grid5000.fr] cat > /tmp/hosts << EOF
hercule-1.lyon.grid5000.fr
hercule-2.lyon.grid5000.fr
hercule-3.lyon.grid5000.fr
hercule-4.lyon.grid5000.fr
EOF
[command][hercule-2.lyon.grid5000.fr] cat > /tmp/hosts << EOF
hercule-1.lyon.grid5000.fr
hercule-2.lyon.grid5000.fr
hercule-3.lyon.grid5000.fr
hercule-4.lyon.grid5000.fr
EOF
[command][hercule-3.lyon.grid5000.fr] cat > /tmp/hosts << EOF
hercule-1.lyon.grid5000.fr
hercule-2.lyon.grid5000.fr
hercule-3.lyon.grid5000.fr
hercule-4.lyon.grid5000.fr
EOF
[command][hercule-4.lyon.grid5000.fr] cat > /tmp/hosts << EOF
hercule-1.lyon.grid5000.fr
hercule-2.lyon.grid5000.fr
hercule-3.lyon.grid5000.fr
hercule-4.lyon.grid5000.fr
EOF
---- hercule-1.lyon.grid5000.fr ----------------------------------------------------------------------------------------------------

{"hercule-4.lyon.grid5000.fr"=>"", "hercule-1.lyon.grid5000.fr"=>"", "hercule-2.lyon.grid5000.fr"=>"", "hercule-3.lyon.grid5000.fr"=>""}

In [32]:
n = 100
h = on roles('mynode'), {:user => 'root', :ssh => {:keys => ['/tmp/sshkey']}} do 
  cmd = "fping -C #{n} -q -s -e -f /tmp/hosts"
end

Connected to hercule-1.lyon.grid5000.fr...
Connected to hercule-3.lyon.grid5000.fr...
Connected to hercule-4.lyon.grid5000.fr...
Connected to hercule-2.lyon.grid5000.fr...
[command][hercule-2.lyon.grid5000.fr] fping -C 100 -q -s -e -f /tmp/hosts
[command][hercule-3.lyon.grid5000.fr] fping -C 100 -q -s -e -f /tmp/hosts
[command][hercule-4.lyon.grid5000.fr] fping -C 100 -q -s -e -f /tmp/hosts
[command][hercule-1.lyon.grid5000.fr] fping -C 100 -q -s -e -f /tmp/hosts
[stderr][hercule-1.lyon.grid5000.fr] hercule-1.lyon.grid5000.fr : 0.01 0.01 0.00 0.01 0.00 0.01 0.01
[stderr][hercule-1.lyon.grid5000.fr]  0.00 0.00 0.00 0.00 0.01 0.00 0.00 0.00 0.00 0.00 0.01 0.01 0.01 0.00 0.00 0.00 0.01 0.01 0.00 0.00 0.00 0.00 0.01 0.00 0.00 0.00 0.01 0.01 0.01
[stderr][hercule-1.lyon.grid5000.fr]  0.00 0.00 0.00 0.01 0.00 0.01 0.01 0.00 0.00 0.01 0.00 0.01 0.01
[stderr][hercule-1.lyon.grid5000.fr]  0.00 0.00 0.01 0.01 0.01 0.01 0.01 0.01 0.01 0.01 0.01 0.01 0.00 0.00 0.00 0.00 0.01
[stderr][hercule-1.lyo

{"hercule-1.lyon.grid5000.fr"=>"hercule-1.lyon.grid5000.fr : 0.01 0.01 0.00 0.01 0.00 0.01 0.01 0.00 0.00 0.00 0.00 0.01 0.00 0.00 0.00 0.00 0.00 0.01 0.01 0.01 0.00 0.00 0.00 0.01 0.01 0.00 0.00 0.00 0.00 0.01 0.00 0.00 0.00 0.01 0.01 0.01 0.00 0.00 0.00 0.01 0.00 0.01 0.01 0.00 0.00 0.01 0.00 0.01 0.01 0.00 0.00 0.01 0.01 0.01 0.01 0.01 0.01 0.01 0.01 0.01 0.01 0.00 0.00 0.00 0.00 0.01 0.01 0.00 0.00 0.00 0.01 0.01 0.01 0.00 0.00 0.00 0.00 0.01 0.00 0.00 0.00 0.00 0.00 0.01 0.00 0.01 0.01 0.00 0.00 0.01 0.01 0.01 0.01 0.01 0.00 0.01 0.01 0.00 0.00 0.01\nhercule-2.lyon.grid5000.fr : 0.04 0.03 0.03 0.03 0.03 0.03 0.03 0.03 0.03 0.03 0.03 0.03 0.03 0.03 0.03 0.03 0.03 0.03 0.03 0.03 0.03 0.03 0.03 0.03 0.03 0.03 0.03 0.03 0.03 0.03 0.03 0.03 0.03 0.03 0.03 0.03 0.03 0.03 0.03 0.03 0.03 0.04 0.04 0.04 0.03 0.03 0.03 0.03 0.03 0.03 0.03 0.04 0.05 0.06 0.04 0.05 0.06 0.06 0.06 0.04 0.05 0.03 0.03 0.03 0.03 0.03 0.03 0.03 0.03 0.03 0.03 0.03 0.03 0.03 0.03 0.03 0.03 0.03 0.03 0.03 0.03 0.06

In [33]:
hh = []
h.each do |k,v|
    hh.concat(h[k]
        .split("\n")[0..3]
        .map{|x| x.split(" ")}
        .map{|a| [k, a[0], a[2..-1].map{|l| l.to_f/n}.sum]})
end
hh

[["hercule-1.lyon.grid5000.fr", "hercule-1.lyon.grid5000.fr", 0.0048], ["hercule-1.lyon.grid5000.fr", "hercule-2.lyon.grid5000.fr", 0.03410000000000005], ["hercule-1.lyon.grid5000.fr", "hercule-3.lyon.grid5000.fr", 0.030600000000000075], ["hercule-1.lyon.grid5000.fr", "hercule-4.lyon.grid5000.fr", 0.030100000000000078], ["hercule-2.lyon.grid5000.fr", "hercule-1.lyon.grid5000.fr", 0.04400000000000003], ["hercule-2.lyon.grid5000.fr", "hercule-2.lyon.grid5000.fr", 0.0015000000000000005], ["hercule-2.lyon.grid5000.fr", "hercule-3.lyon.grid5000.fr", 0.030000000000000075], ["hercule-2.lyon.grid5000.fr", "hercule-4.lyon.grid5000.fr", 0.030200000000000077], ["hercule-3.lyon.grid5000.fr", "hercule-1.lyon.grid5000.fr", 0.030400000000000076], ["hercule-3.lyon.grid5000.fr", "hercule-2.lyon.grid5000.fr", 0.030100000000000078], ["hercule-3.lyon.grid5000.fr", "hercule-3.lyon.grid5000.fr", 0.0006000000000000001], ["hercule-3.lyon.grid5000.fr", "hercule-4.lyon.grid5000.fr", 0.030100000000000075], ["her

In [34]:
require 'nyaplot'

false

In [35]:

x=hh.map{|x| x[0].match('\w*-(\d*)')[1].to_i}
y=hh.map{|x| x[1].match('\w*-(\d*)')[1].to_i}
fill=hh.map{|x|x[2]}


[0.0048, 0.03410000000000005, 0.030600000000000075, 0.030100000000000078, 0.04400000000000003, 0.0015000000000000005, 0.030000000000000075, 0.030200000000000077, 0.030400000000000076, 0.030100000000000078, 0.0006000000000000001, 0.030100000000000075, 0.04410000000000006, 0.030200000000000074, 0.030000000000000075, 0.0005]

In [36]:
plot5 = Nyaplot::Plot.new
hm = plot5.add(:heatmap, x, y, fill)
hm.stroke_color("#fff")
hm.stroke_width("0")
hm.width(1)
hm.height(1)
plot5.legend(true)
plot5.show

#<Nyaplot::Frame:0x00000004e523e8 @properties={:panes=>[#<Nyaplot::Plot:0x00000004e585e0 @properties={:diagrams=>[#<Nyaplot::Diagram:0x00000004e52a78 @properties={:type=>:heatmap, :options=>{:x=>"data0", :y=>"data1", :fill=>"data2", :stroke_color=>"#fff", :stroke_width=>"0", :width=>1, :height=>1}, :data=>"f1398d8b-d346-43ef-bd78-89e13e6349bf"}, @xrange=[1, 4], @yrange=[1, 4]>], :options=>{:legend=>true, :zoom=>true, :width=>800, :xrange=>[1, 4], :yrange=>[1, 4]}}>], :data=>{"f1398d8b-d346-43ef-bd78-89e13e6349bf"=>#<Nyaplot::DataFrame:0x00000004e58018 @name="f1398d8b-d346-43ef-bd78-89e13e6349bf", @rows=[{:data0=>1, :data1=>1, :data2=>0.0048}, {:data0=>1, :data1=>2, :data2=>0.03410000000000005}, {:data0=>1, :data1=>3, :data2=>0.030600000000000075}, {:data0=>1, :data1=>4, :data2=>0.030100000000000078}, {:data0=>2, :data1=>1, :data2=>0.04400000000000003}, {:data0=>2, :data1=>2, :data2=>0.0015000000000000005}, {:data0=>2, :data1=>3, :data2=>0.030000000000000075}, {:data0=>2, :data1=>4, :data2=>0.030200000000000077}, {:data0=>3, :data1=>1, :data2=>0.030400000000000076}, {:data0=>3, :data1=>2, :data2=>0.030100000000000078}, {:data0=>3, :data1=>3, :data2=>0.0006000000000000001}, {:data0=>3, :data1=>4, :data2=>0.030100000000000075}, {:data0=>4, :data1=>1, :data2=>0.04410000000000006}, {:data0=>4, :data1=>2, :data2=>0.030200000000000074}, {:data0=>4, :data1=>3, :data2=>0.030000000000000075}, {:data0=>4, :data1=>4, :data2=>0.0005}]>}, :extension=>[]}>

["display_data", "<IDS|MSG>", "f5e38a3985f08f32801bf96314f9a7e2e67319ef079b67357e187310762274ec", "{\"msg_type\":\"display_data\",\"msg_id\":\"a97be377-5e01-4e2f-b6d2-ed24de1085e1\",\"username\":\"kernel\",\"session\":\"75077eb9-0f18-4e63-a060-1996117e4e60\",\"version\":\"5.0\"}", "{\"username\":\"username\",\"version\":\"5.0\",\"msg_id\":\"4FB0D01FFB9B4E7B8F2A8CAE8E9F4538\",\"msg_type\":\"execute_request\",\"session\":\"58C7C83E175F4FFC8397372B9095F801\"}", "{}", "{\"data\":{\"text/html\":\"<div id='vis-78aa4e9f-563f-4f9f-baf2-2885b48f46f2'></div>\\n<script>\\n(function(){\\n    var render = function(){\\n        var model = {\\\"panes\\\":[{\\\"diagrams\\\":[{\\\"type\\\":\\\"heatmap\\\",\\\"options\\\":{\\\"x\\\":\\\"data0\\\",\\\"y\\\":\\\"data1\\\",\\\"fill\\\":\\\"data2\\\",\\\"stroke_color\\\":\\\"#fff\\\",\\\"stroke_width\\\":\\\"0\\\",\\\"width\\\":1,\\\"height\\\":1},\\\"data\\\":\\\"f1398d8b-d346-43ef-bd78-89e13e6349bf\\\"}],\\\"options\\\":{\\\"legend\\\":true,\\\"zoom\\\":

In [ ]:
myxp.clean